# [실습] LangChain Expression Language


LangChain Expression Language(LCEL)는 랭체인에서 체인을 간결하게 구성하는 문법입니다.   
먼저, LCEL에서 체인이 구성되는 기본적인 구조에 대해 알아봅시다.


In [ ]:
!pip install langchain langchain_google_genai

## Gemini API 준비하기


Google API 키를 등록하고 입력합니다.   
구글 계정 로그인 후 https://aistudio.google.com  에 접속하면, API 키 생성이 가능합니다.

In [ ]:
import os
# Google API KEY 설정
os.environ['GOOGLE_API_KEY']="AIxxx"


LCEL의 가장 큰 특징은, Chain의 구성 요소를 **|**  (파이프)로 연결하여 한 번에 실행한다는 점입니다. 예시를 보겠습니다.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# topic에 대한 영어 농담을 하고, 이것이 왜 농담인지 한국어로 설명하세요.
fun_chat_template = ChatPromptTemplate([
    ('user',"""
Tell me an English joke about {topic} that uses a pun.

Then, provide the following in Korean:
1.  An explanation focusing on the pun, detailing why it is funny to English speakers.
2.  A Korean translation of the joke. Please make your best effort to provide an excellent translation
that captures the spirit and wordplay of the original English pun, even if a direct equivalent is difficult to find.""")])

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash',
                           temperature=0.5,
                           max_tokens=2048)

-----------
LCEL의 구조에서는 템플릿과 llm 모델을 설정하고, 이를 하나로 묶어 체인을 생성합니다.

In [ ]:
joke = fun_chat_template | llm
joke

이후, 체인의 invoke를 실행하며 입력 포맷을 전달하면, 순서대로 체인이 실행되며 최종 결과로 연결됩니다.    
입력 포맷은 Dict 형식으로 전달합니다.

In [ ]:
response = joke.invoke({'topic':'eggs'})

response

In [ ]:
print(response.content)

batch()를 실행하면, 여러 개의 데이터를 병렬적으로 전달합니다.

In [ ]:
topic_list = ['cucumber', 'mango', 'peanut']
response = joke.batch(topic_list)
response

## 실습) 매개변수가 2개인 Prompt-LLM Chain 생성하기   
임의의 ChatPromptTemplate를 만들고, 2개의 매개변수를 받도록 구성하여 체인을 만들고 실행하세요.

In [ ]:
prompt = ChatPromptTemplate(
    [
        ('system','당신은 매우 창의적이며 재미있는 이야기꾼입니다.'),
        ('user','{A}와 {B}가 만났을 때의 가상 대화를 써 주세요.')
    ]
)

In [ ]:
chain = prompt | llm

In [ ]:
response = chain.invoke({'A':'햄릿 왕자', 'B':'슈퍼 마리오'})
print(response.content)

<br><br><br><br><br><br><br><br><br><br><br><br>

In [ ]:
prompt = ChatPromptTemplate(
    [
        ('system', '당신은 재미있고 교훈적인 이야기를 씁니다.'),
        ('user', '{A}와 {B}가 만났을 때의 대화를 써 주세요.')
    ])
chain = prompt | llm
response = chain.invoke({'A':'햄릿', 'B':'호머 심슨'})
print(response.content)

<br><br>
## Chain에 파서 추가하기


LCEL의 체인에는 **파서(Parser)** 를 추가할 수 있습니다.    
파서는 출력 형식을 변환합니다.

StrOutputParser : 출력 결과를 String 형식으로 변환합니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

travel_template = ChatPromptTemplate([
    ('system', '당신은 전문적인 여행 플래너입니다.'),
    ('user', '''다음 여행지에 대한 2박 3일의 여행 계획을 만들어주세요: {destination}''')
])

In [ ]:
travel_chain = travel_template | llm | StrOutputParser()
response = travel_chain.invoke({'destination':'서울'})
print(response)

파서는 스트링이 아닌 json 형식으로도 만들 수 있습니다.   
프롬프트에서 형식을 요청하고, 이를 파서와 결합하여 변환하는 방식입니다.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

jsonparser = JsonOutputParser()

In [ ]:
jsonparser.get_format_instructions()

In [ ]:
travel_template = ChatPromptTemplate([
    ('system', '당신은 전문적인 여행 플래너입니다.'),
    ('user', '''다음 여행지에 대한 여행 계획을 만들어주세요: {destination}
    상세한 정보를 JSON 형식으로 제공해주세요.''')
])

travel_chain = travel_template | llm | jsonparser

In [ ]:
response = travel_chain.invoke({'destination':'베니스'})
response

Json으로 파싱하는 방법은 활용도가 높지만, 실행할 때마다 결과뿐만 아니라 형식도 달라진다는 문제가 있습니다.

In [ ]:
response = travel_chain.invoke({'destination':'아이슬란드'})
response

## Pydantic을 이용해 확실한 형식 지정하기

pydantic은 데이터 형식에 제약조건을 두고 이를 준수하는지 검증하는 라이브러리입니다.


In [ ]:
from pydantic import BaseModel, Field
# pydantic 연동

# Pydantic 모델 정의
class TravelPlan(BaseModel):
    destination: str = Field(description="여행지 이름")
    best_season: str = Field(description="최적의 방문 시기")
    duration: str = Field(description="추천 여행 기간")
    must_visit: list[str] = Field(description="필수 방문지 리스트")
    estimated_cost: str = Field(description="예상 비용 (1인 기준)")
    items_to_pack: list[str] = Field(description="준비물 리스트")
    local_foods: list[str] = Field(description="현지 음식 추천")
    tips: list[str] = Field(description="여행 팁과 주의사항")


In [ ]:
parser = JsonOutputParser(pydantic_object=TravelPlan)

In [ ]:
print(parser.get_format_instructions())

In [ ]:
travel_template = ChatPromptTemplate([
    ('system', '당신은 전문적인 여행 플래너입니다.'),
    ('user', '''다음 여행지에 대한 여행 계획을 만들어주세요: {destination}
    상세한 정보를 JSON 형식으로 제공해주세요.
    {format_instructions}
    ''')
])

travel_chain2 = travel_template | llm | parser


In [ ]:
travel_chain2.invoke({'destination':'카이로', 'format_instructions':parser.get_format_instructions()})

partial을 통해 먼저 일부를 입력할 수도 있습니다.

In [ ]:
travel_chain2 = travel_template.partial(format_instructions=parser.get_format_instructions()) | llm | parser

travel_chain2.invoke('제주도')

# Structured Output
LangChain의 Structured_Output 기능을 사용할 수도 있습니다.

In [ ]:
structured_llm = llm.with_structured_output(TravelPlan)
response = structured_llm.invoke("당일치기 전주국제영화제 혼영 플랜 만들어줘.")
response

In [ ]:
# 허깅페이스 오픈 모델의 경우, Structured_Output 기능이 지원되지 않는 경우가 많습니다.

from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object = TravelPlan)

travel_template = ChatPromptTemplate([
    ('system', '당신은 전문적인 여행 플래너입니다.'),
    ('user', '''다음 여행지에 대한 여행 계획을 만들어주세요: {destination}
    상세한 정보를 JSON 형식으로 제공해주세요.
    {format_instructions}
    ''')
])

structured_llm2 = travel_template.partial(format_instructions = parser.get_format_instructions()) | llm | parser

response = structured_llm2.invoke('부산')
response

프롬프트를 잘 구성하거나, Schema를 사용한다면 답글의 형식을 통일할 수 있습니다.

<br><br>
## Runnables

Runnables는 LCEL의 기본 단위로, 입력을 받아 출력을 생성하는 기본 단위입니다.    
llm, prompt, chain 등이 모두 Runnable 구조에 해당합니다.

이번에는, 데이터 흐름을 제어하는 특별한 Runnable인   
RunnablePassthrough와 RunnableParallel을 이용해 체인을 구성해 보겠습니다.


<br><br>
### RunnablePassthrough
RunnablePassthrough는 체인의 직전 출력을 그대로 가져옵니다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate(["{actor}의 대표 작품은 무엇입니까?"])
chain1 = (
    prompt1
    | llm
    | StrOutputParser()
    | {'answer': RunnablePassthrough()})

response = chain1.invoke("브래드 피트")
response

<br><br>
### RunnableParallel

RunnableParallel은 서로 다른 체인을 병렬적으로 실행합니다.

In [ ]:
from langchain_core.runnables import RunnableParallel

prompt1 = ChatPromptTemplate(["임의의 색깔을 하나 출력하세요. 색깔만 출력하세요."])
prompt2 = ChatPromptTemplate(["임의의 음식을 하나 출력하세요, 음식만 출력하세요."])

chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()

chain3 = RunnableParallel(color = chain1, food = chain2)

chain3.invoke({})


<br><br><br>이번에는 LLM의 결과를 다음 LLM으로 연결해 보겠습니다.

In [ ]:
prompt1 = ChatPromptTemplate(["내쉬빌은 어느 나라의 도시입니까?"])
prompt2 = ChatPromptTemplate(
    ["{country}의 대표적인 인물 3명을 나열하세요. 인물의 이름만 출력하세요."]
)

chain1 = prompt1 | llm | StrOutputParser()
chain2 =(
    {"country": chain1} | prompt2 | llm | StrOutputParser()
)
chain2.invoke({})

RunnableParallel.assign을 통해, 중간 체인인 chain1의 결과와 chain2의 결과를 함께 얻을 수 있습니다.   

In [ ]:
prompt1 = ChatPromptTemplate(["내쉬빌은 어느 나라의 도시입니까?"])
prompt2 = ChatPromptTemplate(
    ["{country}의 대표적인 인물 3명을 나열하세요. 인물의 이름만 출력하세요."]
)


chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()

chain3 = RunnableParallel(country = chain1).assign(people = chain2)

chain3.invoke({})

chain2에서 새로운 매개변수가 추가되는 경우는 어떻게 해야 할까요?

Lambda 함수를 통해, 입력 dict로부터 값을 선택합니다.

In [ ]:
prompt1 = ChatPromptTemplate(["{city}는 어느 나라의 도시인가요? 나라 이름만 출력하세요."])
prompt2 = ChatPromptTemplate(["{country}의 유명한 인물은 누가 있나요? {num} 명의 이름을 나열하세요. 사람 이름만 ,로 구분하여 나열하세요."])

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    RunnableParallel(country = chain1, num = lambda x:x['num'])
    # lambda x:f(x) --> x가 주어지면 f(x)를 return
    | prompt2
    | llm
    | StrOutputParser()
)

print(chain2.invoke({"city": "내쉬빌", "num": "3"}))

<br><br>
체인을 분리하고 RunnableParallel을 이용하면 중간 과정을 모두 출력할 수 있습니다.

In [ ]:
chain4 = (prompt2
    | llm
    | StrOutputParser())

chain3 = RunnableParallel(country = chain1, num = lambda x:x['num']).assign(res = chain4)

chain3.invoke({"city": "부에노스 아이레스", "num": "3"})

JsonOutputParser를 쓴다면 아래와 같이 만들 수도 있습니다.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

prompt1 = ChatPromptTemplate(
    ["전혀 관련이 없는 임의의 단어 두 개를 출력하세요. 출력은 json 형식으로 하세요. 각 항목은 word1, word2로 표시하세요."])
prompt2 = ChatPromptTemplate(["{word1}와 {word2}의 공통점에 대한 10문장 길이의 글을 작성하세요."])

chain1 = prompt1 | llm | JsonOutputParser()
chain2 =(
     chain1 | prompt2 | llm | StrOutputParser()
)
print(chain2.invoke({}))

In [ ]:
chain3 = prompt2 | llm | StrOutputParser()

chain4 = chain1.assign(article=chain3)

chain4.invoke({})